Obtain the parameters used for scalar decomposition in GLV algorithm.
Method described in [this note](
https://hackmd.io/@drouyang/glv)
and [this paper](https://www.iacr.org/archive/crypto2001/21390189.pdf)

In [1]:
def ext_euclidean(a, b):
    assert a <= b
    sqr_b = sqrt(b)
    u = a
    v = b
    x1 = 1
    y1 = 0
    x2 = 0
    y2 = 1

    X = []
    Y = []
    R = []

    flag = -1
    i = -1
    while not u == 0:
        q = v // u
        r = v - q * u
        if flag == -1:
            if r < sqr_b:
                flag = i # exactly the largest index i such that R[i] >= sqrt_b
        i = i + 1
        R.append(r)  #R[i] = r
        x = x2 - q * x1
        X.append(x)
        y = y2 - q * y1  # ax + by = d
        Y.append(y)
        v = u
        u = r
        x2 = x1
        x1 = x
        y2 = y1
        y1 = y
    
    d = v
    x = x2
    y = y2
    X.append(x)
    Y.append(y)
    R.append(0)

    # ((r_l, t_l), (r_l_1, t_l_1), (r_l_2, t_l_2))
    return ((R[flag], X[flag]), (R[flag + 1], X[flag + 1]), (R[flag + 2], X[flag + 2]))  

In [21]:
# Helper functions
def u_64_little_endian(n):
    str_hex = hex(n)
    str_hex_without_0x = str_hex[2:]
    full_width_str = '0' * (112 - len(str_hex_without_0x)) + str_hex_without_0x
    assert len(full_width_str) == 112

    res = []
    for i in range(7):
        temp = '0x' + full_width_str[112 - 16 * (i + 1) : 112 - 16 * i]
        res.append(temp)
    return res

def print_for_rust(elem):
    print( hex(elem))
    print('\n')
    print('[')
    for limb in u_64_little_endian(elem):
        print(limb + ",")
    print(']')
    print('------\n')

In [30]:
def GLV(scalar, n):
    ((r_l, t_l), (r_l_1, t_l_1), (r_l_2, t_l_2)) = ext_euclidean(scalar, n)
    b_1 = t_l_1
    a_1 = r_l_1
    if (r_l^2 + t_l^2) <= (r_l_2^2 + t_l_2^2):
        b_2 = -t_l
        a_2 = r_l
    else:
        b_2 = -t_l_2
        a_2 = r_l_2

    return ((a_1, b_1), (a_2, b_2))  # two points: V1 and V2

def get_endo_params(n):
    Fn = GF(n)
    zeta = Fn.zeta(3)
    ( (_, b1), (_, b2)) = GLV(int(zeta), n)
    gamma1 = round(b1 * 2^448 /n)
    gamma2 = round(b2 * 2^448/ n)

    return (b1, b2, gamma1, gamma2)
    

In [31]:
# Eris scalar field
q_str = "0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5cda8a6c7be4a7a5fe8fadffd6a2a7e8c30006b9459ffffcd300000001"
q = int(q_str, 16)

endo_params = get_endo_params(q)
for c in endo_params:
    print_for_rust(c)

0x60000000000030000196800005ff0065a001ae513ffffde200000001


[
0x3ffffde200000001,
0x05ff0065a001ae51,
0x0000300001968000,
0x0000000060000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
]
------

0x8000000000002000010f00000000


[
0x2000010f00000000,
0x0000800000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
]
------

0x2aaaaaaaaaaa955554a0aaaab2aae415b8e5c9500df52222a6a19d565


[
0xdf52222a6a19d565,
0x2aae415b8e5c9500,
0xaaa955554a0aaaab,
0x00000002aaaaaaaa,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
]
------

0x38e38e38e38e0e38e224e38e4e39d


[
0xe38e224e38e4e39d,
0x00038e38e38e38e0,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
]
------



In [32]:
# Pluto scalar field
p_str = "0x24000000000024000130e0000d7f70e4a803ca76f439266f443f9a5c7a8a6c7be4a775fe8e177fd69ca7e85d60050af41ffffcd300000001"
p = int(q_str, 16)

endo_params = get_endo_params(p)
for c in endo_params:
    print_for_rust(c)

0x60000000000030000196800005ff0065a001ae513ffffde200000001


[
0x3ffffde200000001,
0x05ff0065a001ae51,
0x0000300001968000,
0x0000000060000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
]
------

0x8000000000002000010f00000000


[
0x2000010f00000000,
0x0000800000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
]
------

0x2aaaaaaaaaaa955554a0aaaab2aae415b8e5c9500df52222a6a19d565


[
0xdf52222a6a19d565,
0x2aae415b8e5c9500,
0xaaa955554a0aaaab,
0x00000002aaaaaaaa,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
]
------

0x38e38e38e38e0e38e224e38e4e39d


[
0xe38e224e38e4e39d,
0x00038e38e38e38e0,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
0x0000000000000000,
]
------

